In [1]:
import tensorcircuit as tc
from tensorcircuit.cloud import apis
from tensorcircuit.noisemodel import NoiseConf
from tensorcircuit.noisemodel import circuit_with_noise
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import curve_fit

n = 9
shots = 8192   
np.set_printoptions(suppress=True, precision=6)

def get_A(l, r):

    A = np.zeros((2 ** (r - l), 2 ** (r - l)))

    for i in range(0, (2 ** (r - l))) :

        circ = tc.Circuit(n)

        for j in range(0, r - l):
            if ((i & (2 ** j)) != 0) :
                circ.x(r - 1 - j)

        res = circ.sample(shots, format = "count_dict_bin", readout_error=readout_error, allow_state=True) 

        for str,c in res.items():
            A[int(str[l : r], 2)][i] += c / shots

    return A

def get_readout_error(n):
    base_error = [0.95, 0.9]
    readout_error = []

    for j in range(n):
        random_change_0 = 0.01 * np.random.rand()
        random_change_1 = 0.01 * np.random.rand()
        readout_error.append([base_error[0] - random_change_0, base_error[1] - random_change_1])

    return readout_error 

readout_error = get_readout_error(n)

A_plus = np.ones((1, 1))
# A_plus = np.kron(A_plus, np.linalg.pinv(get_A(0, 4)))
# A_plus = np.kron(A_plus, np.linalg.pinv(get_A(4, 8)))
# A_plus = np.kron(A_plus, np.linalg.pinv(get_A(8, 9)))
A_plus = np.kron(A_plus, np.linalg.pinv(get_A(0, 9)))

circ = tc.Circuit(n)
circ.h(1)
circ.h(3)
circ.x(4)
circ.cz(1,3)
circ.cx(3,5)
circ.h(8)
print(circ.draw())

res = circ.sample(shots, format = "count_dict_bin", allow_state=True)

real_x = np.zeros(2 ** n)
for str,c in res.items():
    real_x[int(str, 2)] = c / shots
print(real_x)

readout_error = get_readout_error(n)

res = circ.sample(shots, format = "count_dict_bin", readout_error=readout_error, allow_state=True)

y = np.zeros(2 ** n)
for str,c in res.items():
    y[int(str, 2)] = c / shots
print(y)

x = A_plus @ y

def function(xf, x):
    return np.linalg.norm(xf - x)

constraints = ({'type': 'eq', 'fun': lambda xf: np.sum(xf) - 1})
bounds = [(0, None) for _ in x]  

initial_guess = np.full_like(x, 1/len(x))
result = minimize(function, initial_guess, args=(x,), constraints=constraints, bounds=bounds, method='SLSQP')
#ZNE Code___
def circuit_with_noise(circ, noise_level):
    noise_conf = NoiseConf(
        amplitude_damping=(noise_level * 0.1, 0.1), 
        phase_damping=(noise_level * 0.1, 0.1)
    )
    noisy_circ = circuit_with_noise(circ, noise_conf)
    res = noisy_circ.sample(shots, format="count_dict_bin", allow_state=True)
    y = np.zeros(2 ** n)
    for str, c in res.items():
        y[int(str, 2)] = c / shots
    return y

def zero_noise_extrapolation(results, noise_levels):
    def linear_func(x, a, b):
        return a * x + b

    popt, _ = curve_fit(linear_func, noise_levels, results)
    return popt[1] 
noise_scaling_factors = [1, 2, 3]


results = []
for noise_level in noise_scaling_factors:
    y_noisy = circuit_with_noise(circ, noise_level)
    y_mitigated_noisy = mitigate_measurement_errors(y_noisy, calib_matrix)
    x_noisy = A_plus @ y_mitigated_noisy
    results.append(x_noisy)

ans_x = result.x
print(ans_x)

zne_results = [np.linalg.norm(x - real_x) for x in results]
zero_noise_estimate = zero_noise_extrapolation(zne_results, noise_scaling_factors)
print("Zero-noise estimate:", zero_noise_estimate)




                  
q_0: ─────────────
     ┌───┐        
q_1: ┤ H ├─■──────
     └───┘ │      
q_2: ──────┼──────
     ┌───┐ │      
q_3: ┤ H ├─■───■──
     ├───┤     │  
q_4: ┤ X ├─────┼──
     └───┘   ┌─┴─┐
q_5: ────────┤ X ├
             └───┘
q_6: ─────────────
                  
q_7: ─────────────
     ┌───┐        
q_8: ┤ H ├────────
     └───┘        
[0.       0.       0.       0.       0.       0.       0.       0.
 0.       0.       0.       0.       0.       0.       0.       0.
 0.125732 0.123291 0.       0.       0.       0.       0.       0.
 0.       0.       0.       0.       0.       0.       0.       0.
 0.       0.       0.       0.       0.       0.       0.       0.
 0.       0.       0.       0.       0.       0.       0.       0.
 0.       0.       0.       0.       0.       0.       0.       0.
 0.125366 0.125732 0.       0.       0.       0.       0.       0.
 0.       0.       0.       0.       0.       0.       0.       0.
 0.       0.       0.       0.      

TypeError: NoiseConf.__init__() got an unexpected keyword argument 'amplitude_damping'